In [39]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [40]:
df=pd.read_csv('wine_fraud.csv')
df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,type
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,Legit,red
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,Legit,red
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,Legit,red
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,Legit,red
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,Legit,red


In [41]:
df.isnull().sum()

fixed acidity           0
volatile acidity        0
citric acid             0
residual sugar          0
chlorides               0
free sulfur dioxide     0
total sulfur dioxide    0
density                 0
pH                      0
sulphates               0
alcohol                 0
quality                 0
type                    0
dtype: int64

In [42]:
df['type'] = pd.get_dummies(df['type'],drop_first=True)

In [43]:
X=df.drop('quality',axis=1)
y=df['quality']

In [44]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [45]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
clf = make_pipeline(StandardScaler(), SVC(class_weight='balanced'))
clf.fit(X_train,y_train)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('svc', SVC(class_weight='balanced'))])

In [46]:
from sklearn.model_selection import GridSearchCV
param_grid = {'svc__C':[0.001,0.01,0.1,0.5,1],
             'svc__kernel':['linear','rbf','poly'],
              'svc__gamma':['scale','auto']}
grid_search=GridSearchCV(clf,param_grid,cv=5,scoring='accuracy')
grid_search.fit(X_train,y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('standardscaler', StandardScaler()),
                                       ('svc', SVC(class_weight='balanced'))]),
             param_grid={'svc__C': [0.001, 0.01, 0.1, 0.5, 1],
                         'svc__gamma': ['scale', 'auto'],
                         'svc__kernel': ['linear', 'rbf', 'poly']},
             scoring='accuracy')

In [47]:
grid_search.best_params_

{'svc__C': 0.001, 'svc__gamma': 'scale', 'svc__kernel': 'poly'}

In [48]:
from sklearn.metrics import confusion_matrix,classification_report
grid_pred = grid_search.predict(X_test)
confusion_matrix(y_test,grid_pred)

array([[   9,   40],
       [  49, 1202]], dtype=int64)

In [49]:
print(classification_report(y_test,grid_pred))

              precision    recall  f1-score   support

       Fraud       0.16      0.18      0.17        49
       Legit       0.97      0.96      0.96      1251

    accuracy                           0.93      1300
   macro avg       0.56      0.57      0.57      1300
weighted avg       0.94      0.93      0.93      1300

